<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/Cleaning_Preprocessingand_adding_columns_with_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting data

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 52.3MB/s]
 84% 49.0M/58.3M [00:02<00:00, 11.7MB/s]
100% 58.3M/58.3M [00:02<00:00, 20.6MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 108MB/s]
 94% 49.0M/52.2M [00:01<00:00, 20.8MB/s]
100% 52.2M/52.2M [00:01<00:00, 33.8MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 155MB/s]


In [2]:
import warnings
warnings.filterwarnings('ignore') 

Loading drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


Reading datasets

In [4]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
isFraud=trn['isFraud']
trn=trn.drop(['isFraud'],1)
tst=pd.read_csv('test_transaction.csv.zip')

Feature selection

In [5]:
useful_cols=list(trn.iloc[:,:55])
useful_cols.remove('TransactionID')
trn=trn[useful_cols]
tst=tst[useful_cols]

Creating unique id

In [6]:
trn['day']=trn['TransactionDT']//86400
tst['day']=tst['TransactionDT']//86400
trn['id']=''
tst['id']=''
trn['d_1']=trn['day']-trn['D1']
tst['d_1']=tst['day']-tst['D1']
for col in ['ProductCD','P_emaildomain','addr1','card1','d_1']:
  trn['id']+=trn[col].astype(str)
  tst['id']+=tst[col].astype(str)
id_trn=trn['id']
id_tst=tst['id']

seperate columns in continuous and categorical

In [7]:
categorical=['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain']+['M'+str(i) for i in range(1,10)]
continuous=[i for i in useful_cols if i not in categorical]

Preprocessing

In [8]:
class ContinuousConverter:
  def __init__(self,name,log_transform,feature):
    self.name=name
    self.skew=feature.skew()
    self.log_transform=log_transform
  def transform(self,feature):
    if self.skew>1:
      feature=self.log_transform(feature)
    mean=feature.mean()
    std=feature.std()
    feature=(feature-mean)/(std+1e-6)
    return feature

In [9]:
from tqdm import tqdm
continuous_trn=[]
continuous_tst=[]
id_cols=[]
for col in tqdm(continuous):
  id_cols+=[col+'_mean',col+'_std']
  trn[col+'_mean']=trn.groupby(['id'])[col].transform('mean')
  trn[col+'_std']=trn.groupby(['id'])[col].transform('std')


  tst[col+'_mean']=tst.groupby(['id'])[col].transform('mean')
  tst[col+'_std']=tst.groupby(['id'])[col].transform('std')
continuous+=id_cols
for col in tqdm(continuous):
  log=lambda x:np.log10(x+(1-min(0,min(x))))
  converter=ContinuousConverter(col,log,trn[col])
  continuous_trn.append(converter.transform(trn[col]).astype(np.float32))
  continuous_tst.append(converter.transform(tst[col]).astype(np.float32))
continuous_trn=pd.DataFrame(continuous_trn).T
continuous_trn.columns=continuous
continuous_tst=pd.DataFrame(continuous_tst).T
continuous_tst.columns=continuous

100%|██████████| 102/102 [00:13<00:00,  7.49it/s]


In [10]:
continuous_trn['isna_sum']=continuous_trn.isna().sum(axis=1)
continuous_tst['isna_sum']=continuous_tst.isna().sum(axis=1)
mn=continuous_trn['isna_sum'].mean()
std=continuous_trn['isna_sum'].std()
continuous_trn['isna_sum']=(continuous_trn['isna_sum']-mn)/std
mn=continuous_tst['isna_sum'].mean()
std=continuous_tst['isna_sum'].std()
continuous_tst['isna_sum']=(continuous_tst['isna_sum']-mn)/std

In [11]:
isna_cols=[]
for col in continuous:
  isna=continuous_trn[col].isna()
  if isna.sum()>0:
    continuous_trn[col+'_isna']=isna.astype(int)
    continuous_tst[col+'_isna']=continuous_tst[col].isna().astype(int)
  continuous_trn[col]=continuous_trn[col].fillna(continuous_trn[col].mean())
  continuous_tst[col]=continuous_tst[col].fillna(continuous_tst[col].mean())

Handling categorical data

In [12]:
from sklearn.preprocessing import OneHotEncoder
def handle_categorical(df_trn,df_tst,categorical,n_values):
  all=[]
  df_trn=df_trn[categorical]
  df_tst=df_tst[categorical]
  for col in categorical:
    all.append(list(df_trn[col].value_counts().iloc[:n_values-1].index)+['Other'])
    df_trn[col]=df_trn[col].map(lambda x:x if x in all[-1] else 'Other')
    df_tst[col]=df_tst[col].map(lambda x:x if x in all[-1] else 'Other')
  ohe=OneHotEncoder(categories=all)
  ohe.fit(pd.concat([df_trn,df_tst]))
  df_trn=pd.DataFrame(ohe.transform(df_trn).toarray()).astype(np.float16)
  df_tst=pd.DataFrame(ohe.transform(df_tst).toarray()).astype(np.float16)
  return df_trn,df_tst
df_trn,df_tst=handle_categorical(trn,tst,categorical,50)
df_trn.shape,df_tst.shape

((590540, 444), (506691, 444))

Concatenating continuous and categorical

In [13]:
trn=pd.concat([df_trn,continuous_trn],1)
tst=pd.concat([df_tst,continuous_tst],1)

Columns created

In [14]:
trn['isFraud']=isFraud
import warnings
warnings.filterwarnings('ignore')


tst.to_csv(F"/content/gdrive/My Drive/fraud/test_id.csv")
trn.to_csv(F"/content/gdrive/My Drive/fraud/train_id.csv")